# Lab-05 Logistic Regression

In [5]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
## 이건 ver1에서 사용되는 enable_eager_execution()을 사용하는 문장
## tf.enable_eager_execution must be called at program startup. 오류 나오면 커널 재시작 하면 된다!
tf.compat.v1.enable_eager_execution()
#tf.executing_eagerly()
tf.random.set_seed(777)
print(tf.__version__)

2.5.0


## 로지스틱 회귀 실행
### 데이터셋 설정

In [2]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print(x_train.shape, y_train.shape)
print(xy)

(759, 8) (759, 1)
[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


In [3]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))
W = tf.Variable(tf.zeros([8,1]), name='weight')
b = tf.Variable(tf.zeros([1]),name = 'bias')

##로지스틱 회귀
def logistic_regression(features):
    #가정 1/1+e^(-x)
    hypothesis  = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis
## 비용함수
def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(logistic_regression(features)) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost
##optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) ver1에서 사용
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

## 정확도(0.5 기준 0과 1리턴)
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) #tf.cast 메서드를 이용해 0.5보다 큰 hypothesis를 입력값으로 받고 이를 float형태로 바꿔줌.
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

## 기울기 계산
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features),features, labels)
    return tape.gradient(loss_value, [W,b])

In [6]:
EPOCHS = 1000
for step in range(EPOCHS+1):
    for features, labels in iter(dataset):
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
        if step%100 ==0:
            print("Iter:{}, Loss:{:.4f}".format(step, loss_fn(logistic_regression(features),features, labels)))

Iter:0, Loss:0.5734
Iter:100, Loss:0.5691
Iter:200, Loss:0.5652
Iter:300, Loss:0.5614
Iter:400, Loss:0.5579
Iter:500, Loss:0.5545
Iter:600, Loss:0.5513
Iter:700, Loss:0.5483
Iter:800, Loss:0.5454
Iter:900, Loss:0.5427
Iter:1000, Loss:0.5402
